In [ ]:
import numpy as np
 
import torch
import torch.nn.functional as F
import torch.nn as nn
 
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
 
import matplotlib.pyplot as plt
 
import operator
from collections import Counter

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
seed = 1234
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

In [ ]:

train_labels_count = np.zeros(47)       # Array to count labels in training dataset
test_labels_count = np.zeros(47)        # Array to count labels in test dataset

train_labels_total = 0                  # Total label count for training dataset
test_labels_total = 0                   # Total label count for test dataset



loss_epochs = []                        # List to store model's training loss for each epoch
loss_batches = []                       # List to store model's training loss for each batch



train_correct_preds = np.zeros(4)       # Array to store count of correct predictions made by models on the training dataset
test_correct_preds = np.zeros(4)        # Array to store count of correct predictions made by models on the test dataset




unsure_labels_count = np.zeros(47)      # Array to count unsure labels predicted by the model on the training dataset
top_unsure_labels = []

unsure_matrix = []                      # Matrix to store unsure label values for each label
for i in range(47):
    unsure_matrix.append([])




train_wrong_preds = np.zeros(47)        # Array to count wrong predictions made by the model, for each label in the training dataset
test_wrong_preds = np.zeros(47)         # Array to count wrong predictions made by the model, for each label in the test dataset



####    Task 4, Q1      ####
loss_epoch_matrix = []                  # Matrix to store training loss per epoch for each model
models_params = np.zeros(4)             # Array to store total parameters for each model
model_txt = ["Model #0", "Model #1", "Model #2", "Model #3"]                    # List of model names for plots

In [ ]:
transformCustom = transforms.Compose([transforms.ToTensor(),                        # convert image to a tensor
                                      transforms.Lambda(lambda x:x.view(-1))])      # flatten 28*28 images into (784,) vector


train = datasets.EMNIST(root='.', split='bymerge', train=True, transform=transformCustom, download=True)
test = datasets.EMNIST(root='.', split='bymerge', train=False, transform=transformCustom, download=True)

  0%|          | 0/561753746 [00:00<?, ?it/s]

Extracting ./EMNIST/raw/gzip.zip to ./EMNIST/raw


KeyboardInterrupt: ignored

In [ ]:
train_loader = DataLoader(train, batch_size=128, shuffle=True)
test_loader = DataLoader(test, batch_size=128)

In [ ]:
# count the instances for each label
for label in train.train_labels:
    train_labels_count[label.item()] += 1

# get the total count of labels for training set
train_labels_total = train_labels_count.sum()

In [ ]:
# count the instances for each label
for label in test.test_labels:
    test_labels_count[label.item()] += 1

# get the total count of labels for test set
test_labels_total = test_labels_count.sum()

In [ ]:
print("-"*20, " For Training Dataset ", "-"*20)

print("\n\n--Number of instances for each class--\n")
for id, n in enumerate(train_labels_count):
    print(f'Label {id}:\t{int(n)}\t\t{n *100 / train_labels_total :.2f} %')

In [ ]:
print("-"*20, " For Test Dataset ", "-"*20)

print("\n\n--Number of instances for each class--\n")
for id, n in enumerate(test_labels_count):
    print(f'Label {id}:\t{int(n)}\t\t{n *100 / test_labels_total :.2f} %')

**Reflection:**

The data set is not balanced as there are 47 classes 

100/47 = 2.127 

that means each class should have a 2.12% - 2.13 % representation , which is not the case here 

the under represent classes are :
10-17, 19, 20, 22, 23, 25-27, 29, 31-38, 40-45

and the over represent classses are :
0-9, 21, 24, 28, 30, 39, 46


# `Q2`

Orignal model has 1703983 trainable parameters

Model : Sequential

Layer  |    Output Shape  |    Parameters




---


Linear    |  (None ,2048)  |    1607680

---


Activation(ReLu)|   (None, 2048)  |  0

---


Dropout    |  (None, 2048)      |  0

---


Linear       |(None, 47)      |   96303

---


Activation(Softmax) | (None, 47)  | 0



Total params: 1,703,983

Trainable params: 1,703,983

Non-trainable params: 0




In [ ]:
class MLP (nn.Module):
    def __init__(self):
        super(MLP, self).__init__()

        self.input = nn.Linear(784, 2048)        #1024  #576    #368
        self.output = nn.Linear(2048, 47)        #1024  #192    #368
        self.drop = nn.Dropout(0.2)


    def forward(self, x):
        x = self.drop(F.relu(self.input(x)))
        x = self.output(x)

        if not self.training:
            x = F.softmax(x, dim=1)

        return x

In [ ]:
class MLP1 (nn.Module):
    def __init__(self):
        super(MLP1, self).__init__()

        self.input = nn.Linear(784, 2048)
        self.output = nn.Linear(2048, 47)


    def forward(self, x):
        x = F.relu(self.input(x))
        x = self.output(x)

        if not self.training:
            x = F.softmax(x, dim=1)
        
        return x

In [ ]:
class MLP2 (nn.Module):
    def __init__(self):
        super(MLP2, self).__init__()

        self.input = nn.Linear(784, 1024)
        self.output = nn.Linear(1024, 47)
        self.drop = nn.Dropout(0.2)

    def forward(self, x):
        x = self.drop(F.relu(self.input(x)))
        x = self.output(x)

        if not self.training:
            x = F.softmax(x, dim=1)

        return x

In [ ]:
class MLP3 (nn.Module):
    def __init__(self):
        super(MLP3, self).__init__()

        self.input = nn.Linear(784, 2048)
        self.layer1 = nn.Linear(2048, 1000)
        self.output = nn.Linear(1000, 47)
        self.drop = nn.Dropout(0.2)

    def forward(self, x):
        x = self.drop(F.relu(self.input(x)))
        x = self.drop(F.relu(self.layer1(x)))
        x = self.output(x)

        if not self.training:
            x = F.softmax(x, dim=1)

        return x

In [ ]:
# Instantiate the MLP
model = MLP()
model_1 = MLP1()
model_2 = MLP2()
model_3 = MLP3()

# Use GPU, if available
model.to(device)
model_1.to(device)
model_2.to(device)
model_3.to(device)

In [ ]:
for p in model.parameters():
    param = p.shape
    if len(param)==2:
        models_params[0] += param[0] * param[1]
    else:
        models_params[0] += param[0]

for p in model_1.parameters():
    param = p.shape
    if len(param)==2:
        models_params[1] += param[0] * param[1]
    else:
        models_params[1] += param[0]

for p in model_2.parameters():
    param = p.shape
    if len(param)==2:
        models_params[2] += param[0] * param[1]
    else:
        models_params[2] += param[0]

for p in model_3.parameters():
    param = p.shape
    if len(param)==2:
        models_params[3] += param[0] * param[1]
    else:
        models_params[3] += param[0]

In [ ]:
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters())
opt_1 = torch.optim.Adam(model_1.parameters())
opt_2 = torch.optim.Adam(model_2.parameters())
opt_3 = torch.optim.Adam(model_3.parameters())

In [ ]:
# set model state to training
model.train()

for epoch in range(10):
    # variable to store loss for current epoch
    loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)   # Use GPU, if available
        opt.zero_grad() #zero the gradients
        predict_batch = model(inputs) #fwd
        loss_batch = loss_fn(predict_batch,labels) #loss
        loss_batch.backward() #backward

        opt.step()#optimize i.e update weights

        # Add current batch's model loss to the list
        loss_batches.append(loss_batch.item())

        # Add current batch's model loss, to calculate loss for current epoch    
        loss += loss_batch.item()
    
    # Add current epoch's model loss to the list
    loss_epochs.append(loss)

# Add this model's training loss per epoch to the matrix
loss_epoch_matrix.append(loss_epochs)

In [ ]:
# Empty the list
loss_epochs = []

# set model state to training
model_1.train()

for epoch in range(10):
    # variable to store loss for current epoch
    loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)   # Use GPU, if available
        opt_1.zero_grad() #zero the gradients
        predict_batch = model_1(inputs) #fwd
        loss_batch = loss_fn(predict_batch,labels) #loss
        loss_batch.backward() #backward

        opt_1.step()#optimize i.e update weights

        # Add current batch's model loss, to calculate loss for current epoch    
        loss += loss_batch.item()
    
    # Add current epoch's model loss to the list
    loss_epochs.append(loss)

# Add this model's training loss per epoch to the matrix
loss_epoch_matrix.append(loss_epochs)

In [ ]:
# Empty the list
loss_epochs = []

# set model state to training
model_2.train()

for epoch in range(10):
    # variable to store loss for current epoch
    loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)   # Use GPU, if available
        opt_2.zero_grad() #zero the gradients
        predict_batch = model_2(inputs) #fwd
        loss_batch = loss_fn(predict_batch,labels) #loss
        loss_batch.backward() #backward

        opt_2.step()#optimize i.e update weights

        # Add current batch's model loss, to calculate loss for current epoch    
        loss += loss_batch.item()
    
    # Add current epoch's model loss to the list
    loss_epochs.append(loss)

loss_epoch_matrix.append(loss_epochs)

In [ ]:
# Empty the list
loss_epochs = []

# set model state to training
model_3.train()

for epoch in range(10):
    # variable to store loss for current epoch
    loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)   # Use GPU, if available
        opt_3.zero_grad() #zero the gradients
        predict_batch = model_3(inputs) #fwd
        loss_batch = loss_fn(predict_batch,labels) #loss
        loss_batch.backward() #backward

        opt_3.step()#optimize i.e update weights

        # Add current batch's model loss, to calculate loss for current epoch    
        loss += loss_batch.item()
    
    # Add current epoch's model loss to the list
    loss_epochs.append(loss)

# Add this model's training loss per epoch to the matrix
loss_epoch_matrix.append(loss_epochs)

In [ ]:
# Change model state to eval
model.eval()
model_1.eval()
model_2.eval()
model_3.eval()


for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)   # use GPU, if available

    # Get model predictions for current batch sample
    y_pred = model(inputs)
    y_pred_1 = model_1(inputs)
    y_pred_2 = model_2(inputs)
    y_pred_3 = model_3(inputs)

    # Get output labels for current batch samples
    _, output = torch.max(y_pred, 1)                        
                                                            
    _, output_1 = torch.max(y_pred_1, 1)
    _, output_2 = torch.max(y_pred_2, 1)
    _, output_3 = torch.max(y_pred_3, 1)



    
    # Get total count of correct predictions
    train_correct_preds[0] += torch.sum(output == labels).item()
    train_correct_preds[1] += torch.sum(output_1 == labels).item()
    train_correct_preds[2] += torch.sum(output_2 == labels).item()
    train_correct_preds[3] += torch.sum(output_3 == labels).item()



   
    # Get count of wrong predictions for each label
    for i in range(len(labels)):
        if output[i] != labels[i]:
            train_wrong_preds[labels[i].item()] += 1



   
    for sample_idx, label in enumerate(labels):
        # Get scalar value
        op_label = label.item()

        # Get label's prediction probability
        label_pred_value = y_pred[sample_idx][op_label].item()

        # Check if the label probability is less than 0.6 (unsure prediction)
        if label_pred_value < 0.6:

            # Store y_pred[sample_idx] as list of tuple (label, probability value)
            label_prob_list = [(label_val, prob) for label_val, prob in enumerate(y_pred[sample_idx].tolist())]

            # Sort the list by the probability value in descending order
            label_prob_list.sort(key = lambda tup: tup[1], reverse = True)

            
            if label_prob_list[0][0] != op_label:
                top_unsure = label_prob_list[0]         # If the max predicted probability's label is not the same as the ground truth label,
                                                        # just use that for top unsure
            else:
                # Get tuple (label, probability) for the first unsure label less than the label probability `label_pred_value`
                top_unsure = label_prob_list[1] 
            

            # Add the unsure labels to the corresponding label index of the matrix
            unsure_matrix[op_label].append(top_unsure)

            # Increase count of unsure labels
            unsure_labels_count[op_label] += 1

In [ ]:
# Change model state to eval
model.eval()
model_1.eval()
model_2.eval()
model_3.eval()

for inputs, labels in test_loader:
    
    inputs, labels = inputs.to(device), labels.to(device)

    # Get model predictions for current batch sample
    y_pred = model(inputs)
    y_pred_1 = model_1(inputs)
    y_pred_2 = model_2(inputs)
    y_pred_3 = model_3(inputs)

    # Get output labels for current batch samples
    _, output = torch.max(y_pred, 1)                       
                                                            
    _, output_1 = torch.max(y_pred_1, 1)
    _, output_2 = torch.max(y_pred_2, 1)
    _, output_3 = torch.max(y_pred_3, 1)


    
    # Get total count of correct predictions
    test_correct_preds[0] += torch.sum(output == labels).item()
    test_correct_preds[1] += torch.sum(output_1 == labels).item()
    test_correct_preds[2] += torch.sum(output_2 == labels).item()
    test_correct_preds[3] += torch.sum(output_3 == labels).item()


   
    # Get count of wrong predictions for each label
    for i in range(len(labels)):
        if output[i] != labels[i]:
            test_wrong_preds[labels[i].item()] += 1


# Change model state to training
model.train()
model_1.train()
model_2.train()
model_3.train()

In [ ]:
plt.plot(range(len(loss_epoch_matrix[0])), loss_epoch_matrix[0])
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.show()

In [ ]:
plt.plot(range(len(loss_batches)), loss_batches)
plt.xlabel('Minibatches')
plt.ylabel('Cross Entropy')
plt.show()

In [ ]:
print("Training:")
print("number of data ", int(train_labels_total))
print("number of wrongly predicted label ", int(train_labels_total - train_correct_preds[0]))
print(f"accuracy {train_correct_preds[0] * 100 / train_labels_total :.2f}%")

print("\nTest:")
print("number of data ", int(test_labels_total))
print("number of wrongly predicted label ", int(test_labels_total - test_correct_preds[0]))
print(f"accuracy {test_correct_preds[0] * 100 / test_labels_total :.2f}%")

In [ ]:
for unsure_labels in unsure_matrix:
    counter = Counter(elem[0] for elem in unsure_labels)
    max_val = max(counter.items(), key=operator.itemgetter(1))[0]
    sum = 0
    for label, prob in unsure_labels:
        if label==max_val:
            sum+=prob
    top_unsure_labels.append([max_val, counter.get(max_val), sum/counter.get(max_val)])

In [ ]:
print("Unsure predictions on training dataset\n")
print("Label\t   #training samples\t#times unsure\t\tpercentage %\t\ttop unsure label, x\t\t#times unsure with x\t\tAveraged P(x)")
for id, n in enumerate(train_labels_count):
    if unsure_labels_count[id] < 10000:
        print(f'{id}\t\t{int(n)}\t\t   {int(unsure_labels_count[id])}\t\t\t   {unsure_labels_count[id] * 100 / n :.2f}%\t\t\t{top_unsure_labels[id][0]}\t\t\t\t{top_unsure_labels[id][1]}\t\t\t   {top_unsure_labels[id][2] :.2f}')
    else:
        print(f'{id}\t\t{int(n)}\t\t   {int(unsure_labels_count[id])}\t\t   {unsure_labels_count[id] * 100 / n :.2f}%\t\t\t{top_unsure_labels[id][0]}\t\t\t\t{top_unsure_labels[id][1]}\t\t\t   {top_unsure_labels[id][2] :.2f}')

 label *40* has the highest percentage of unsure predictions.

In [ ]:
print("Prediction error on training dataset\n")

print("Label\t\t# sample\twrongly predicted\tprediction error by labels (%)\t\tprediction error overall")
for i in range(47):
    print(f'{i}\t\t  {int(train_labels_count[i])}\t\t\t{int(train_wrong_preds[i])}\t\t\t{train_wrong_preds[i] *100 / train_labels_count[i] :.2f}\t\t\t\t\t{train_wrong_preds[i] *100 / train_labels_total :.2f}')

In [ ]:
print("Prediction error on test dataset\n\n")
print("Label\t\t# sample\twrongly predicted\tprediction error by labels (%)\t\tprediction error overall")
for i in range(47):
    print(f'{i}\t\t  {int(test_labels_count[i])}\t\t\t{int(test_wrong_preds[i])}\t\t\t{test_wrong_preds[i] *100 / test_labels_count[i] :.2f}\t\t\t\t\t{test_wrong_preds[i] *100 / test_labels_total :.2f}')

***Reflections***


Label *40* has the highest prediction error by label %.  

This does agree with the unsure prediction results, as label 40 also has the highest percentage of unsure predictions as well.

In [ ]:
plt.plot(range(len(loss_epoch_matrix[0])), loss_epoch_matrix[0], label='Original model')
plt.plot(range(len(loss_epoch_matrix[1])), loss_epoch_matrix[1], label='Same  w/o Dropout')
plt.plot(range(len(loss_epoch_matrix[2])), loss_epoch_matrix[2], label='Same  w/ less parameters')
plt.plot(range(len(loss_epoch_matrix[3])), loss_epoch_matrix[3], label='1 more layer')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.title('Training Loss')
plt.legend()
plt.show()

In [ ]:
model_train_accuracy = [x*100/train_labels_total for x in train_correct_preds]
model_test_accuracy = [x*100/test_labels_total for x in test_correct_preds]

model_train_error = [(100 - x)/100 for x in model_train_accuracy]
model_test_error = [(100 - x)/100 for x in model_test_accuracy]

In [ ]:
plt.plot(models_params, model_train_accuracy, marker='x', color='b', label='Training')
plt.plot(models_params, model_test_accuracy, marker='^', color='r', label='Test')
plt.xlabel('Model Parameters')
plt.ylabel('Prediction Accuracy %')
plt.title('Training and Test Accuracy')

for i, txt in enumerate(model_txt):
    plt.annotate(txt, (models_params[i], model_train_accuracy[i]))
    plt.annotate(txt, (models_params[i], model_test_accuracy[i]))

plt.legend()
plt.show()

In [ ]:
plt.plot(models_params, model_train_error, marker='x', color='b', label='Training')
plt.plot(models_params, model_test_error, marker='^', color='r', label='Test')
plt.xlabel('Model Parameters')
plt.ylabel('Prediction Errors')
plt.title('Training and Test Error')

for i, txt in enumerate(model_txt):
    plt.annotate(txt, (models_params[i], model_train_error[i]))
    plt.annotate(txt, (models_params[i], model_test_error[i]))

plt.legend()
plt.show()